### Making notebook for the processing of text, eaiser to work with rather than constantly opening
Will prob only need to use once, just need that juicy data

In [1]:
import time
import requests
from bs4 import BeautifulSoup as Bs
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
# Site to scrape the train station data, only need to run once
headers = {
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) '
                       'Chrome/75.0.3770.142 Safari/537.36'
    }

# run when using selenium
driver = webdriver.Chrome()


station_url = "https://en.wikipedia.org/wiki/List_of_Metro_Trains_Melbourne_railway_stations"
# webscraping doesn't seem to work, trying to pen the site
driver.get(station_url)
time.sleep(1)
html_content = driver.page_source
raw_html = Bs(html_content, 'html.parser')

# print(raw_html)
table_data = raw_html.find("table", class_="wikitable sortable mw-collapsible sticky-header jquery-tablesorter mw-made-collapsible")
# table_data = soup.findAll("table")
rows = table_data.find_all("tr")
print(table_data.find("tbody"))
# rows[0] is the header column, typically ignore, just need the titles

# Headers for data
"""
Name

Image

Transport
connections


Service(s)

Distance from Southern Cross[4]

Zone(s)

Date opened[5]

Suburb

Notes
[5]
"""

# if rows:
#     print(rows)

<tbody>
<tr>
<th scope="row"><a href="/wiki/Aircraft_railway_station" title="Aircraft railway station">Aircraft</a>
</th>
<td><span typeof="mw:File/Frameless"><a class="mw-file-description" href="/wiki/File:Aircraft_Station_(2).jpg"><img alt="Southbound view from platform 1 at Aircraft" class="mw-file-element" data-file-height="3263" data-file-width="4351" decoding="async" height="75" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Aircraft_Station_%282%29.jpg/100px-Aircraft_Station_%282%29.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Aircraft_Station_%282%29.jpg/150px-Aircraft_Station_%282%29.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Aircraft_Station_%282%29.jpg/200px-Aircraft_Station_%282%29.jpg 2x" width="100"/></a></span>
</td>
<td><span typeof="mw:File"><a href="/wiki/Aircraft_railway_station#Transport_links" title="Aircraft railway station"><img alt="Aircraft railway station#Transport links" class="mw-file-element" data-file-height

'\nName\n\nImage\n\nTransport\nconnections\n\n\nService(s)\n\nDistance from Southern Cross[4]\n\nZone(s)\n\nDate opened[5]\n\nSuburb\n\nNotes\n[5]\n'

In [7]:
table_contents = table_data.find("tbody")
rows = table_contents.find_all("tr")

In [71]:
def remove_brackets(text):
    # Remove digits within square brackets
    return re.sub(r'\[\d+\]', '', text)

In [85]:
# For each given row,
# Name
# Image of station
# Transport connections (Bus, tram etc)
# Service(s) (Train lines)
# Distance from Southern Cross, km and mi
# Zone(s) (1, 2, both or vline)
# Date opened
# Suburb
# Notes
station_data_header = {"station_name": 0, "image_of_station": 1, "transport_connection": 2, "services":3,
                       "distance_from_sX_km":4, "distance_from_sX_mi":5, "zone":6, "date_opened":7, "suburb":8, "notes":9}

bus_image = "//upload.wikimedia.org/wikipedia/en/thumb/4/41/Victoria_bus_logo.svg/30px-Victoria_bus_logo.svg.png"
tram_image = "//upload.wikimedia.org/wikipedia/en/thumb/e/ef/Melbourne_tram_logo.svg/30px-Melbourne_tram_logo.svg.png"
skybus_image = "//upload.wikimedia.org/wikipedia/en/thumb/3/39/Melbourne_skybus_logo.svg/30px-Melbourne_skybus_logo.svg.png"
coach_image = "//upload.wikimedia.org/wikipedia/en/thumb/9/9b/Victoria_coach_logo.svg/30px-Victoria_coach_logo.svg.png"
# Note how text ends with \n

df = pd.DataFrame

for station_data in rows:
    temp_data = []
    
    temp_data.append(station_data.find("th").text)
    
    # for each of the rows, get data
    row_data = station_data.find_all("td")
    
    # Getting image of station
    img_tag = row_data[0].find('img')
    if img_tag:
        img_src = img_tag.get('src')
        temp_data.append(img_src)
    else:
        print("No img tag found.")
        temp_data.append(None)
    
    # transport connections
    transport_methods = []
    for transport in row_data[1].find_all("img"):
        current_transport = transport.get("src")
        # Not the most efficient but it works
        if current_transport == bus_image:
            transport_methods.append("bus")
        elif current_transport == tram_image:
            transport_methods.append("tram")
        elif current_transport == skybus_image:
            transport_methods.append("skybus")
        elif current_transport == coach_image:
            transport_methods.append("coach_image")
    temp_data.append(transport_methods)
    
    # services
    temp_data.append(row_data[2].text)

    # distance
    # km
    temp_data.append(row_data[3].text)
    # mi
    temp_data.append(row_data[4].text)

    # zone
    zone_raw = row_data[5].text
    zone_processed = remove_brackets(zone_raw)
    if "/" in zone_processed:
        temp_data.append(zone_processed.split("/"))
    else:
        temp_data.append(zone_processed)
    
    # date opened
    temp_data.append(row_data[6].text)
    
    # suburb
    suburb = remove_brackets(row_data[7].text)
    temp_data.append(suburb)
    
    # notes
    text = remove_brackets(row_data[8].text)
    temp_data.append(text)

    break

df = pd.DataFrame(rows, columns=station_data_header.keys())

# Set 'station_name' as the index
df.set_index('station_name', inplace=True)


['Aircraft\n',
 '//upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Aircraft_Station_%282%29.jpg/100px-Aircraft_Station_%282%29.jpg',
 ['bus'],
 'Werribee\n',
 '22.23\n',
 '13.81\n',
 '2\n',
 '7 March 1925\n',
 'Laverton\n',
 'Rebuilt in 2022\n']

In [74]:
remove_brackets("1/2[216]").split("/")

['1', '2']